**EDGS COLAB:** https://colab.research.google.com/github/CompVis/EDGS/blob/main/notebooks/fit_model_to_scene_full.ipynb#scrollTo=5b039ea9-e9ea-41a7-b487-cc799a3ebf85

In [ ]:
# Clone repo and initialize submodules
!git clone https://github.com/Parskatt/RoMa.git
!git clone https://github.com/camenduru/simple-knn.git
!git clone https://github.com/CompVis/EDGS.git --recursive
!git clone https://github.com/graphdeco-inria/gaussian-splatting.git
!git clone https://github.com/graphdeco-inria/diff-gaussian-rasterization.git

In [2]:
import sys, os

if 'scene' in sys.modules:
    del sys.modules['scene']  # Remove any existing 'scene' from sys.modules to avoid cached imports

sys.path.insert(0, "/content/EDGS")
sys.path.insert(0, "/content/RoMa")
sys.path.insert(0, "/content/simple-knn")
sys.path.insert(0, "/content/gaussian-splatting")
sys.path.insert(0, "/content/diff-gaussian-rasterization")

In [ ]:
!nvcc --version

In [ ]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

In [ ]:
import os
print(os.getcwd())

In [ ]:
%cd /content/diff-gaussian-rasterization

In [ ]:
# Make sure third_party folder exists
!mkdir -p third_party

# Get GLM from official repo
!git clone https://github.com/g-truc/glm.git third_party/glm

In [ ]:
!pip install -e .

In [ ]:
%cd /content

**Testig the Running of Gaussian Rasterization Installation**

In [11]:
import torch
from diff_gaussian_rasterization import GaussianRasterizer, GaussianRasterizationSettings

In [ ]:
# Example: make dummy rasterization settings
settings = GaussianRasterizationSettings(
    image_height=512,
    image_width=512,
    tanfovx=1.0,
    tanfovy=1.0,
    bg=torch.zeros(3, device="cuda"),
    scale_modifier=1.0,
    viewmatrix=torch.eye(4, device="cuda"),
    projmatrix=torch.eye(4, device="cuda"),
    sh_degree=0,
    campos=torch.zeros(3, device="cuda"),
    prefiltered=False,
    debug=False
)

rasterizer = GaussianRasterizer(settings)
print("GaussianRasterizer ready!")

**Testing Close**

In [ ]:
!pip install wandb hydra-core tqdm torchmetrics lpips matplotlib rich plyfile

**Install ROMA**

In [ ]:
%cd /content/RoMa

In [ ]:
!pip install -e .

In [ ]:
%cd ..

In [ ]:
import os
print(os.getcwd())

**Install Other Dependencies**

In [ ]:
!pip install pycolmap

In [ ]:
!pip install wandb hydra-core tqdm torchmetrics lpips matplotlib rich plyfile imageio imageio-ffmpeg

In [ ]:
!pip install gradio

In [ ]:
!pip install plotly scikit-learn moviepy==2.1.1 ffmpeg

In [ ]:
!pip install opencv-python

**Install Simple_KNN**

In [ ]:
%cd /content/simple-knn

In [ ]:
!pip install -e .

In [ ]:
%cd ..

In [ ]:
import os
print(os.getcwd())

**Import Libraries & Configuration file**

In [27]:
# Check that scene/__init__.py exists here
assert os.path.exists("/content/gaussian-splatting/scene/__init__.py")
assert os.path.exists("/content/EDGS/configs/train.yaml")

In [ ]:
%cd /content/EDGS

In [ ]:
import os
print(os.getcwd())

In [30]:
import wandb
import hydra
import torch
import random
import shutil
import omegaconf
import numpy as np
%matplotlib inline
# %matplotlib ipympl
from omegaconf import OmegaConf
from matplotlib import pyplot as plt
from hydra import initialize, compose

In [31]:
from source.utils_aux import set_seed
from source.trainer import EDGSTrainer

In [ ]:
with initialize(config_path="../content/EDGS/configs", version_base="1.1"):
    cfg = compose(config_name="train")

print(OmegaConf.to_yaml(cfg))

**Process Video**

In [ ]:
PATH_TO_VIDEO = "/content/EDGS/assets/examples/video_fruits.mp4"
num_ref_views = 100 # how many frames you want to extract from video and colmap
num_corrs = 10000   # number of feature correspondences

if True:
    # Full pipeline helpers
    from gradio_demo import preprocess_input
    from source.utils_preprocess import read_video_frames, preprocess_frames, select_optimal_frames, save_frames_to_scene_dir, run_colmap_on_scene
    from source.visualization import generate_circular_camera_path, save_numpy_frames_as_mp4, generate_fully_smooth_cameras_with_tsp, put_text_on_image
    # Change to your path
    images, scene_dir = preprocess_input(PATH_TO_VIDEO, num_ref_views, num_corrs)
    shutil.copytree(scene_dir, '/content/EDGS/assets/video_colmaped/scene',  dirs_exist_ok=True)

In [ ]:
import os

scene_path = "/content/EDGS/assets/video_colmaped/scene"  # or whatever you set in cfg
for root, dirs, files in os.walk(scene_path):
    level = root.replace(scene_path, "").count(os.sep)
    indent = " " * 4 * (level)
    print(f"{indent}{os.path.basename(root)}/")
    subindent = " " * 4 * (level + 1)
    for f in files:
        print(f"{subindent}{f}")

**Update Config File With your own settings**

In [36]:
cfg.wandb.name="EDGS.demo.scene"
cfg.wandb.mode="disabled" # "online"
cfg.gs.dataset.model_path="./scene_edgsed/"  # "change this to your path to the processed scene"
#cfg.gs.dataset.source_path="../assets/scene_colmaped/" # "change this to your path"

cfg.gs.dataset.source_path="./assets/video_colmaped/scene"  # < --- For Video


cfg.gs.dataset.images="images"
cfg.gs.opt.TEST_CAM_IDX_TO_LOG=12
cfg.train.gs_epochs=30000
cfg.gs.opt.opacity_reset_interval=1_000_000
cfg.train.no_densify=False
cfg.init_wC.matches_per_ref=15_000
cfg.init_wC.nns_per_ref=3
cfg.init_wC.num_refs=180
cfg.init_wC.roma_model="outdoors"

# SH & render
cfg.gs.sh_degree = 3
cfg.gs.opt.batch_size = 2048   # or 1024/2048/4096 if GPU allows
cfg.init_wC.add_SfM_init = True
cfg.gs.opt.densify_from_iter = 1000
cfg.gs.opt.densify_until_iter = 20000
cfg.gs.opt.densification_interval = 800
cfg.gs.dataset.white_background = False
cfg.gs.pipe.convert_SHs_python = True
cfg.gs.pipe.compute_cov3D_python = True

# **Initialize Model and Logger**

In [ ]:
!wandb login

In [40]:
!mkdir -p /root/.cache/torch/hub/checkpoints
!cp /content/drive/MyDrive/roma_outdoor.pth /root/.cache/torch/hub/checkpoints/

In [ ]:
_ = wandb.init(entity=cfg.wandb.entity,
                   project=cfg.wandb.project,
                   config=omegaconf.OmegaConf.to_container(cfg, resolve=True, throw_on_missing=True),
                   name = cfg.wandb.name,
                   mode = cfg.wandb.mode)
omegaconf.OmegaConf.resolve(cfg)
set_seed(cfg.seed)
# Init output folder
print("Output folder: {}".format(cfg.gs.dataset.model_path))
os.makedirs(cfg.gs.dataset.model_path, exist_ok=True)
# Init gs model
gs = hydra.utils.instantiate(cfg.gs)
trainer = EDGSTrainer(GS=gs,
                      training_config=cfg.gs.opt,
                      device=cfg.device)

**Init With Matchings**

In [ ]:
trainer.timer.start()
trainer.init_with_corr(cfg.init_wC)
trainer.timer.pause()

In [ ]:
with torch.no_grad():
    viewpoint_stack = trainer.GS.scene.getTrainCameras()
    viewpoint_cams_to_viz = random.sample(trainer.GS.scene.getTrainCameras(), 4)
    for viewpoint_cam in viewpoint_cams_to_viz:
        render_pkg = trainer.GS(viewpoint_cam)
        image = render_pkg["render"]

        image_np = image.clone().detach().cpu().numpy().transpose(1, 2, 0)
        image_gt_np = viewpoint_cam.original_image.clone().detach().cpu().numpy().transpose(1, 2, 0)

        # Clip values to be in the range [0, 1]
        image_np = np.clip(image_np*255, 0, 255).astype(np.uint8)
        image_gt_np = np.clip(image_gt_np*255, 0, 255).astype(np.uint8)

        fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(12,6))
        ax[0].imshow(image_gt_np)
        ax[0].axis("off")
        ax[1].imshow(image_np)
        ax[1].axis("off")
        plt.tight_layout()
        plt.show()

# **Optimize The Scene**

Optimize first briefly for 5k steps and visualize results. We also disable saving of pretrained models. Train function can be changed for any other method

In [ ]:
trainer.saving_iterations = []
cfg.train.gs_epochs=5000
trainer.train(cfg.train)

In [ ]:
with torch.no_grad():
    for viewpoint_cam in viewpoint_cams_to_viz:
        render_pkg = trainer.GS(viewpoint_cam)
        image = render_pkg["render"]

        image_np = image.clone().detach().cpu().numpy().transpose(1, 2, 0)
        image_gt_np = viewpoint_cam.original_image.clone().detach().cpu().numpy().transpose(1, 2, 0)

        # Clip values to be in the range [0, 1]
        image_np = np.clip(image_np*255, 0, 255).astype(np.uint8)
        image_gt_np = np.clip(image_gt_np*255, 0, 255).astype(np.uint8)

        fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(12,6))
        ax[0].imshow(image_gt_np)
        ax[0].axis("off")
        ax[1].imshow(image_np)
        ax[1].axis("off")
        plt.tight_layout()
        plt.show()

In [ ]:
with torch.no_grad():
    trainer.save_model()

**Continue training until we reach total 30K training steps**

In [ ]:
trainer.saving_iterations = []
cfg.train.gs_epochs=15_000
trainer.train(cfg.train)

In [ ]:
with torch.no_grad():
    for viewpoint_cam in viewpoint_cams_to_viz:
        render_pkg = trainer.GS(viewpoint_cam)
        image = render_pkg["render"]

        image_np = image.clone().detach().cpu().numpy().transpose(1, 2, 0)
        image_gt_np = viewpoint_cam.original_image.clone().detach().cpu().numpy().transpose(1, 2, 0)

        # Clip values to be in the range [0, 1]
        image_np = np.clip(image_np*255, 0, 255).astype(np.uint8)
        image_gt_np = np.clip(image_gt_np*255, 0, 255).astype(np.uint8)

        fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(12,6))
        ax[0].imshow(image_gt_np)
        ax[0].axis("off")
        ax[1].imshow(image_np)
        ax[1].axis("off")
        plt.tight_layout()
        plt.show()

In [ ]:
with torch.no_grad():
    trainer.save_model()

In [ ]:
import shutil
import os

folder_path = "/content/EDGS/assets/video_colmaped"

# Check if the folder exists before deleting
if os.path.exists(folder_path) and os.path.isdir(folder_path):
    shutil.rmtree(folder_path)
    print(f"Folder '{folder_path}' deleted successfully.")
else:
    print(f"Folder '{folder_path}' does not exist.")

In [ ]:
import shutil

folder_path = "/content/EDGS/scene_edgsed/point_cloud"
zip_path = "/content/gs_PointCloud_1.zip"

# Create a zip file
shutil.make_archive("/content/gs_PointCloud_1", 'zip', folder_path)